In [ ]:
import os.path
import datetime
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import netCDF4
%matplotlib widget
from matplotlib import pyplot
from matplotlib import dates

setup_dir = "."
sst_da_dir = "."
varname = "total_chlorophyll_calculator_result"
cmap = 'viridis'          # color map to use for filled contour plots [alternative: cmocean.cm.thermal]

In [ ]:
# Read satellite observations of sea surface temperature
obs = []
if varname == 'temp':
    for l in open(os.path.join(setup_dir, 'cci_sst.dat')):
        if not l.startswith('#'):
            dt, value, sd = l.rstrip().split('\t')
            obs.append([datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'), float(value), float(sd)])
if varname == 'total_chlorophyll_calculator_result':
    for l in open(os.path.join(setup_dir, 'cci_chl.dat')):
        if not l.startswith('#'):
            dt, value, sd = l.rstrip().split('\t')
            mu = float(value)
            sigma = float(sd)
            obs.append([datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'), 10.**mu, 10.**(mu-0.67448*sigma), 10.**(mu+0.67448*sigma)])
obs = np.array(obs).reshape(-1, 4)

## Plot reference GOTM results

Forecast-only, no data assimilation

In [ ]:
nc = netCDF4.Dataset(os.path.join(setup_dir, 'result.nc'))

time = netCDF4.num2date(nc['time'], nc['time'].units)
mpltime = dates.date2num(time)
z = -nc['z'][:, :, 0, 0]
ncvar = nc[varname]
ref = ncvar[:, :, 0, 0]
ref_sf = ref[:, -1]

fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

ax1.plot_date(obs[:,0], obs[:,1], '.k', label='satellite')
ax1.plot_date(mpltime, ref_sf, '-', color='C0', label='model')
ax1.set_xlim(mpltime[0], mpltime[-1])
ax1.set_ylabel(f'{ncvar.long_name} ({ncvar.units})')
ax1.grid()
ax1.legend()
ax1.set_title(f'surface {ncvar.long_name}')
cax1.axis('off')

mpltime_2d = np.broadcast_to(mpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z, ref, 20)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label(f'{ncvar.long_name} ({ncvar.units})')
ax2.set_ylabel('depth (m)')
ax2.set_title(ncvar.long_name)
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())
ax2.grid()

fig.tight_layout()

## Plot ensemble GOTM results

In [ ]:
# Load ensemble results
N = 20
ncs = [netCDF4.Dataset(os.path.join(setup_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]
ncs_sst_only = [netCDF4.Dataset(os.path.join(sst_da_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]

enstime = netCDF4.num2date(ncs[0]['time'], ncs[0]['time'].units)
ensmpltime = dates.date2num(enstime)
z = -ncs[0].variables['z'][:, :, 0, 0]
ens = [nc[varname][:, :, 0, 0] for nc in ncs]
ens_sf = [temp[:, -1] for temp in ens]

In [ ]:
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(figsize=(10,10), nrows=3, ncols=2, sharex='col', width_ratios=[0.95, 0.05], height_ratios=[0.4, 0.3, 0.3])
low = obs[:,1] - obs[:,2]
high = obs[:,3] - obs[:,1]
#ax1.plot_date(obs[:,0], obs[:,1], '.k', alpha=0.4, label='satellite')
ax1.errorbar(obs[:,0], obs[:,1], yerr=[low, high], ecolor='k', elinewidth=1., fmt='.k', alpha=0.4, zorder=-1, label='satellite')
ax1.plot_date(time, ref_sf, '-', color='C0', label='model, no DA')
if setup_dir != sst_da_dir:
    ens_phys_only = [nc[varname][:, :, 0, 0] for nc in ncs_sst_only]
    ens_sf_phys_only = [temp[:, -1] for temp in ens_phys_only]
    ax1.plot_date(enstime, np.mean(ens_sf_phys_only, axis=0), '-', color='C2', label='model, phys DA')
ax1.plot_date(enstime, np.mean(ens_sf, axis=0), '-', color='C1', label='model, phys+bgc DA')
ax1.set_xlim(time[0], time[-1])
#ax1.set_xlim(datetime.datetime(2021,1,1), time[-1])
ax1.set_ylabel(f'chlorophyll ({ncvar.units})')
ax1.grid()
ax1.legend()
ax1.set_title('surface chlorophyll')
ax1.set_ylim(0, 4)
cax1.axis('off')

chl_contours = np.linspace(0., 5., 11)
mpltime_2d = np.broadcast_to(ensmpltime[:, np.newaxis], z.shape)
ens_mean = np.mean(ens, axis=0)
ens_diff = ens_mean - ref[-enstime.size:, :]
pc = ax2.contourf(mpltime_2d, z, ref, chl_contours, cmap=cmap, extend='max')
cb = fig.colorbar(pc, cax=cax2)
cb.set_label(f'chlorophyll ({ncvar.units})')
ax2.set_ylabel('height (m)')
ax2.set_title(f'simulated chlorophyll (no DA)');
ax2.grid()
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())

#pc = ax3.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=np.linspace(-3.5,3.5,21), extend='both')
pc = ax3.contourf(mpltime_2d, z, ens_mean, chl_contours, cmap=cmap, extend='max')
cb = fig.colorbar(pc, cax=cax3)
cb.set_label(f'chlorophyll ({ncvar.units})')
ax3.set_ylabel('height (m)')
ax3.set_title(f'simulated chlorophyll (phys+bgc DA)');
ax3.grid()
ax3.xaxis.axis_date()
ax3.set_ylim(z.max(), z.min())

fig.tight_layout()

In [ ]:
# Plot time variation in biogeochemical parameters - only if estimating these!

estimated_pars = "instances_phy_parameters_mumax0", "instances_dia_parameters_mumax0"

fig, axes = pyplot.subplots(figsize=(8, 4 * len(estimated_pars)), nrows=len(estimated_pars))
for parname, ax in zip(estimated_pars, axes):
    ens = np.array([nc[parname][:, 0, 0] for nc in ncs])
    ax.fill_between(ensmpltime, ens.min(axis=0), ens.max(axis=0), alpha=0.5)
    ax.plot(ensmpltime, ens.mean(axis=0), '-k')
    ax.xaxis.axis_date()
    ncpar = ncs[0][parname]
    ax.set_ylabel(f"{ncpar.long_name} ({ncpar.units})")
    ax.grid()
    ax.set_title(parname)
fig.tight_layout()